In [1]:
import requests
import pandas as pd
from secret import API_KEY

def dfFromAboveDiamond():
    queues = ['challenger', 'grandmaster', 'master']
    list = []
    for i in queues:
        response = requests.get(f"https://euw1.api.riotgames.com/lol/league/v4/{i}leagues/by-queue/RANKED_SOLO_5x5?api_key={API_KEY}")
        json = response.json()['entries']
        df = pd.DataFrame.from_dict(json)
        df.insert(4, 'tier', i)
        list.append(df)

    return pd.concat(list)

def dfFromDiamond1(nbr):
    list = []
    for i in range(1, nbr):
        response = requests.get(f"https://euw1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/DIAMOND/I?page={i}&api_key={API_KEY}")
        json = response.json()
        df = pd.DataFrame.from_dict(json)
        list.append(df)
    
    return pd.concat(list)

In [2]:
high = dfFromAboveDiamond()

In [3]:
low = dfFromDiamond1(2)

In [4]:
df = pd.concat([high, low]).reset_index(drop=True)

In [10]:
df = df.drop(['veteran', 'inactive', 'freshBlood', 'hotStreak'], axis=1)
df = df.dropna(axis=1)
df = df[df['totalgames'] > 30]

In [6]:
df['totalgames'] = df.apply(lambda x: x['wins'] + x['losses'], axis=1)
df['winrate'] = df.apply(lambda x: round(x['wins'] / x['totalgames'] * 100, 1), axis=1)